In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [2]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x



In [3]:
# test model
model = NN(784, 10)
x = torch.rand(64, 784)
model(x).shape

torch.Size([64, 10])

In [4]:
device= torch.device('mps')

In [5]:
# hyperparamerter
input_size = 784
num_classes = 10
batch_size = 32
learning_rate = 0.001
num_epochs = 1


In [20]:
# load data
train_data = datasets.MNIST(root="./mnist/", train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

test_data = datasets.MNIST(root="./mnist/", train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)

In [26]:
# initialize NN
model = NN(input_size=input_size, num_classes=num_classes).to(device)


In [27]:
# Loss and Optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [37]:
# train the network
for epoch in range(num_epochs):
    for batch_idx , (data, targets) in enumerate(train_loader):
        data = data.to(device)
        targets = targets.to(device)
        # get correct shape
        data = data.reshape(data.shape[0], -1) # flattern batch, m,n to batch, n*m
        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradent descent or adam step
        optimizer.step()
        




In [41]:
# calculate accuracy

def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on Training Dataset')
    else:
        print('Checking accuracy on Test Dataset')
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            x = x.reshape(x.shape[0], -1)
            scores = model(x)
            _ , predictions = scores.max(1)
            num_correct += (predictions ==y).sum()
            num_samples += predictions.shape[0]
    acc = float(num_correct)/float(num_samples)*100
    print(f'Got {num_correct} / {num_samples} with accuracy {acc}')

    model.train() # it is used when we use this function in side training loop,
    #so model.eval() stops training, to put model to training mode and continue the training it is set
    return acc

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on Training Dataset
Got 56681 / 60000 with accuracy 94.46833333333333
Checking accuracy on Test Dataset
Got 9426 / 10000 with accuracy 94.26


94.26

In [49]:
# Create simple CNN
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1)) # last 3 parameter have default values 
        self.fc1 = nn.Linear(16*7*7, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)

        return x
    


In [50]:
# test CNN
model = CNN()
x = torch.randn(64, 1, 28, 28)
model(x).shape

torch.Size([64, 10])

In [51]:
# hyperparamerter
in_channels = 1
num_classes = 10
batch_size = 32
learning_rate = 0.001
num_epochs = 1


In [53]:
# initialize CNN
model = CNN().to(device)


In [54]:
# Loss and Optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [55]:
# train the network
for epoch in range(num_epochs):
    for batch_idx , (data, targets) in enumerate(train_loader):
        data = data.to(device)
        targets = targets.to(device)
        
        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradent descent or adam step
        optimizer.step()

In [57]:
# calculate accuracy

def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on Training Dataset')
    else:
        print('Checking accuracy on Test Dataset')
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            scores = model(x)
            _ , predictions = scores.max(1)
            num_correct += (predictions ==y).sum()
            num_samples += predictions.shape[0]
    acc = float(num_correct)/float(num_samples)*100
    print(f'Got {num_correct} / {num_samples} with accuracy {acc}')

    model.train() # it is used when we use this function in side training loop,
    #so model.eval() stops training, to put model to training mode and continue the training it is set
    return acc

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on Training Dataset
Got 58325 / 60000 with accuracy 97.20833333333333
Checking accuracy on Test Dataset
Got 9718 / 10000 with accuracy 97.18


97.18

In [71]:
# hyperparamerter
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
batch_size = 32
learning_rate = 0.001
num_epochs = 2


In [72]:
# Create simple RNN
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size , num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size*sequence_length, num_classes)

        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward Prop
        out, _ = self.rnn(x, h0)
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)

        return out


In [73]:
# initialize RNN
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)


In [74]:
# Loss and Optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [75]:
# train the network
for epoch in range(num_epochs):
    for batch_idx , (data, targets) in enumerate(train_loader):
        data = data.to(device).squeeze(1)
        targets = targets.to(device)
        
        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradent descent or adam step
        optimizer.step()

In [76]:
# calculate accuracy

def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on Training Dataset')
    else:
        print('Checking accuracy on Test Dataset')
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device).squeeze(1)
            y = y.to(device)
            scores = model(x)
            _ , predictions = scores.max(1)
            num_correct += (predictions ==y).sum()
            num_samples += predictions.shape[0]
    acc = float(num_correct)/float(num_samples)*100
    print(f'Got {num_correct} / {num_samples} with accuracy {acc}')

    model.train() # it is used when we use this function in side training loop,
    #so model.eval() stops training, to put model to training mode and continue the training it is set
    return acc

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on Training Dataset
Got 57534 / 60000 with accuracy 95.89
Checking accuracy on Test Dataset
Got 9565 / 10000 with accuracy 95.65


95.65

In [95]:
# hyperparamerter
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
batch_size = 32
learning_rate = 0.001
num_epochs = 7


In [96]:
# Loss and Optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [97]:
# Create simple Bidirectional LSTM
class BLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size , num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2  , num_classes)

        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        # Forward Prop
        out, _ = self.lstm(x, (h0,c0))
        
        out = self.fc(out[:, -1, :])

        return out

In [98]:
# initialize BLSTM
model = BLSTM(input_size, hidden_size, num_layers, num_classes).to(device)


In [104]:
def save_checkpoint(state, filename="mnist_checkpoint.pth.tar"):
    print("Saving checkpoint")
    torch.save(state, filename)

def load_checkpoint(checkpoint):
    print("Loading checkpoint")
    model.load_state_dict(checkpoint['state_dict'])
    #optimizer.load_state_dict(checkpoint['state_dict'])

In [105]:
load_model = True
num_epochs = 1

if load_model:
    load_checkpoint(torch.load("mnist_checkpoint.pth.tar"))

# train the network
for epoch in range(num_epochs):


    for batch_idx , (data, targets) in enumerate(train_loader):
        data = data.to(device).squeeze(1)
        targets = targets.to(device)
        
        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradent descent or adam step
        optimizer.step()
    
    if epoch %2 == 0:
        checkpoint = {
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }
        save_checkpoint(checkpoint)

Loading checkpoint
Saving checkpoint


In [106]:
# calculate accuracy

def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on Training Dataset')
    else:
        print('Checking accuracy on Test Dataset')
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device).squeeze(1)
            y = y.to(device)
            scores = model(x)
            _ , predictions = scores.max(1)
            num_correct += (predictions ==y).sum()
            num_samples += predictions.shape[0]
    acc = float(num_correct)/float(num_samples)*100
    print(f'Got {num_correct} / {num_samples} with accuracy {acc}')

    model.train() # it is used when we use this function in side training loop,
    #so model.eval() stops training, to put model to training mode and continue the training it is set
    return acc
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on Training Dataset
Got 6265 / 60000 with accuracy 10.441666666666666
Checking accuracy on Test Dataset
Got 1028 / 10000 with accuracy 10.280000000000001


10.280000000000001